In [18]:
import pandas as pd
from datetime import datetime

In [11]:
df = pd.read_csv('ETHBTC-1m.csv')

In [24]:
df.head(5)

,open_time,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
0,1609459200000,0.025464,0.025464,0.025447,0.025449,625.182,1609459259999,15.915061,188,225.278,5.734506,0
1,1609459260000,0.025449,0.025479,0.025435,0.025465,243.229,1609459319999,6.193560,171,90.259,2.298146,0
2,1609459320000,0.025465,0.025475,0.025447,0.025454,195.633,1609459379999,4.980014,107,66.517,1.693127,0
3,1609459380000,0.025453,0.025463,0.025442,0.025443,135.177,1609459439999,3.440634,87,59.216,1.507130,0
4,1609459440000,0.025443,0.025450,0.025427,0.025436,308.023,1609459499999,7.834913,115,187.253,4.762954,0


In [12]:
df.describe()

,open_time,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
count,9.389100e+05,938910.000000,938910.000000,938910.000000,938910.000000,938910.000000,9.389100e+05,938910.000000,938910.000000,938910.000000,938910.000000,938910.0
mean,1.637672e+12,0.063267,0.063292,0.063241,0.063267,119.395209,1.637672e+12,6.572083,183.570057,58.873810,3.246211,0.0
std,1.627950e+10,0.014759,0.014760,0.014758,0.014759,185.799018,1.627950e+10,9.480170,224.754577,99.023891,5.115744,0.0
min,1.609459e+12,0.023090,0.023132,0.023083,0.023090,0.000000,1.609459e+12,0.000000,0.000000,0.000000,0.000000,0.0
25%,1.623579e+12,0.058257,0.058285,0.058230,0.058259,30.567500,1.623579e+12,2.089847,75.000000,13.409000,0.908464,0.0
50%,1.637686e+12,0.067978,0.068002,0.067953,0.067978,63.678800,1.637686e+12,3.947388,125.000000,30.172700,1.863984,0.0
75%,1.651770e+12,0.073310,0.073333,0.073287,0.073309,138.833000,1.651770e+12,7.514529,213.000000,67.370000,3.697792,0.0
max,1.665853e+12,0.088330,0.088366,0.088306,0.088332,21933.475900,1.665853e+12,1308.407466,16517.000000,10053.086000,554.255723,0.0


In [31]:
def time_features(df):
    df["utc"] = df.open_time.apply(lambda x: datetime.utcfromtimestamp(x/1000))
    df['minute'] = df.utc.apply(lambda time: time.minute)
    df['hour'] = df.utc.apply(lambda time: time.hour)
    df['day'] = df.utc.apply(lambda time: time.day)
    df["weekday"] = df.utc.apply(lambda time: time.dayofweek)
    df['month'] = df.utc.apply(lambda time: time.month)
    df['year'] = df.utc.apply(lambda time: time.year).astype("category")
    return df

In [33]:
df = time_features(df)

In [38]:
create_feature_dict = {
    'open': [np.sum, np.std, np.max],
}

data = df.groupby([df.year, df.month, df.day, df.hour]).agg().dropna()

In [39]:
data

open_time      open      high       low     close  \
year month day hour                                                         
2021 1     1   0     1.609461e+12  0.025381  0.025390  0.025369  0.025378   
               1     1.609465e+12  0.025351  0.025369  0.025336  0.025353   
               2     1.609468e+12  0.025440  0.025455  0.025425  0.025440   
               3     1.609472e+12  0.025469  0.025478  0.025458  0.025468   
               4     1.609475e+12  0.025414  0.025424  0.025402  0.025413   
...                           ...       ...       ...       ...       ...   
2022 10    15  13    1.665841e+12  0.067081  0.067091  0.067074  0.067084   
               14    1.665844e+12  0.067263  0.067269  0.067257  0.067264   
               15    1.665848e+12  0.067265  0.067269  0.067259  0.067264   
               16    1.665851e+12  0.067224  0.067228  0.067219  0.067223   
               17    1.665853e+12  0.067197  0.067197  0.067195  0.067196   

                         volume    close_time       qav  num_trades  \
year month day hour                                                   
2021 1     1   0     178.749467  1.609461e+12  4.536900  111.350000   
               1     304.732600  1.609465e+12  7.726140  172.933333   
               2     185.894700  1.609468e+12  4.730580  108.333333   
               3     116.465733  1.609472e+12  2.966583   63.116667   
               4     115.408450  1.609475e+12  2.932676   71.500000   
...                         ...           ...       ...         ...   
2022 10    15  13     55.440065  1.665841e+12  3.720085  109.316667   
               14     44.428025  1.665844e+12  2.988831   83.150000   
               15     35.992865  1.665848e+12  2.421363   66.483333   
               16     20.856300  1.665851e+12  1.402006   56.016667   
               17     10.975867  1.665853e+12  0.737536   23.000000   

                     taker_base_vol  taker_quote_vol  ignore  minute  weekday  
year month day hour                                                            
2021 1     1   0          74.649367         1.894888     0.0    29.5      4.0  
               1         179.487583         4.550385     0.0    29.5      4.0  
               2         101.163400         2.574820     0.0    29.5      4.0  
               3          52.471183         1.336806     0.0    29.5      4.0  
               4          53.878683         1.369240     0.0    29.5      4.0  
...                             ...              ...     ...     ...      ...  
2022 10    15  13         42.123148         2.826919     0.0    29.5      5.0  
               14         30.181818         2.030628     0.0    29.5      5.0  
               15         21.840328         1.469402     0.0    29.5      5.0  
               16         11.734743         0.788804     0.0    29.5      5.0  
               17          7.404933         0.497583     0.0     1.0      5.0  

[15653 rows x 14 columns]